The Purpose of this code is examine various data about several ski resort towns in the western U.S. and compare them in order to recommend the best town to buy a vacation or retirement home. We will look at Foursquare venue data to find which towns have the most restaurants, non skiing recreation opportunities, etc. We will also look at a variety of general quality of life data including cost of living index, the weather, and quality of the local schools among others, We will also look at the available skiing experience itself. Finally we will compare the data to the median price per square foot of housing in each town.

In [1]:
#import initially needed libraries and packages
import pandas as pd
import numpy as np

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
#import folium # map rendering library

print('Libraries imported.')


Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


## Create dataframe of non-Foursquare data

In [2]:
data = {'Town': ['Aspen', 'Crested Butte', 'Incline Village', 'Jackson', 'Park City', 'Steamboat Springs', 'Sun Valley', 'Taos','Telluride', 'Vail'],
       'Total Skiable Acreage': [3362, 1547, 21754, 4909, 9326, 3006, 2154, 1494, 2000, 9611],
       'Ski In/Out Acreage': [673, 1547, 655, 2500, 7300, 2956, 2154, 1294, 2000, 5289],
       'Vertical': [4406, 3055, 4139, 3100, 3668, 3400, 3500, 3281, 4425, 3450],
       'Lifts': [38, 16, 162, 54, 62, 20, 18, 20, 18, 79],
       'Avg Snowfall': [300, 300, 400, 459, 355, 349, 220, 300, 309, 549],
       'Avg Jan Temp': [21, 13, 26, 14, 28, 19, 14, 24, 26, 17],
       'Air Quality Index': [41, 41, 44, 40, 47, 43, 13, 15, 43, 42],
       'Avg An Sunny Days': [246, 273, 251, 212, 229, 242, 204, 283, 242, 247],
       'Cost of Living Index': [140, 153, 172, 144, 180, 139, 145, 104, 168, 149],
       'School Quality': [.917, .986, .939, .962, .804, .936, .806, .6, .959, .567],
       'Crime Rate': [.0426, .0178, 0.0111, .0185, .0353, .0247, .0123, .0951, .0238, .0459],
       'Median Home Value (000s)': [1116, 543, 972, 552, 805, 577, 661, 294, 1058, 743],
       'Median Home Price per sq ft': [1647, 521, 533, 823, 652, 429, 498, 226, 1380, 1243],
       'Latitude': [39.1906, 38.8697, 39.2497, 43.4803, 40.6464, 40.4847, 43.6808, 36.4103, 37.9375, 39.6406],
       'Longitude': [-106.8186, -106.9869, -119.9525, -110.7625, -111.4975, -106.8317, -114.3644, -105.573, -107.8119, -106.3741]}


In [3]:
# This data frame contains the skiing related data and the quality of life data.
skitownsdf = pd.DataFrame(data)
skitownsdf

,Town,Total Skiable Acreage,Ski In/Out Acreage,Vertical,Lifts,Avg Snowfall,Avg Jan Temp,Air Quality Index,Avg An Sunny Days,Cost of Living Index,School Quality,Crime Rate,Median Home Value (000s),Median Home Price per sq ft,Latitude,Longitude
0,Aspen,3362,673,4406,38,300,21,41,246,140,0.917,0.0426,1116,1647,39.1906,-106.8186
1,Crested Butte,1547,1547,3055,16,300,13,41,273,153,0.986,0.0178,543,521,38.8697,-106.9869
2,Incline Village,21754,655,4139,162,400,26,44,251,172,0.939,0.0111,972,533,39.2497,-119.9525
3,Jackson,4909,2500,3100,54,459,14,40,212,144,0.962,0.0185,552,823,43.4803,-110.7625
4,Park City,9326,7300,3668,62,355,28,47,229,180,0.804,0.0353,805,652,40.6464,-111.4975
5,Steamboat Springs,3006,2956,3400,20,349,19,43,242,139,0.936,0.0247,577,429,40.4847,-106.8317
6,Sun Valley,2154,2154,3500,18,220,14,13,204,145,0.806,0.0123,661,498,43.6808,-114.3644
7,Taos,1494,1294,3281,20,300,24,15,283,104,0.600,0.0951,294,226,36.4103,-105.5730
8,Telluride,2000,2000,4425,18,309,26,43,242,168,0.959,0.0238,1058,1380,37.9375,-107.8119
9,Vail,9611,5289,3450,79,549,17,42,247,149,0.567,0.0459,743,1243,39.6406,-106.3741


In [4]:
#exploring skitowndf 
skitownsdf.describe()

,Total Skiable Acreage,Ski In/Out Acreage,Vertical,Lifts,Avg Snowfall,Avg Jan Temp,Air Quality Index,Avg An Sunny Days,Cost of Living Index,School Quality,Crime Rate,Median Home Value (000s),Median Home Price per sq ft,Latitude,Longitude
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.00000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,5916.300000,2636.800000,3642.400000,48.700000,354.100000,20.200000,36.900000,242.90000,149.400000,0.847600,0.032710,732.100000,795.200000,39.959060,-109.697310
std,6326.824534,2116.433835,508.670861,45.553509,94.234283,5.613476,12.233379,24.19573,21.375999,0.152502,0.024994,259.446612,469.007415,2.262286,4.555575
min,1494.000000,655.000000,3055.000000,16.000000,220.000000,13.000000,13.000000,204.00000,104.000000,0.567000,0.011100,294.000000,226.000000,36.410300,-119.952500
25%,2038.500000,1357.250000,3310.750000,18.500000,300.000000,14.750000,40.250000,232.25000,141.000000,0.804500,0.017975,558.250000,503.750000,38.949925,-111.313750
50%,3184.000000,2077.000000,3475.000000,29.000000,329.000000,20.000000,41.500000,244.00000,147.000000,0.926500,0.024250,702.000000,592.500000,39.445150,-107.399400
75%,8221.750000,2842.000000,4021.250000,60.000000,388.750000,25.500000,43.000000,250.00000,164.250000,0.954000,0.040775,930.250000,1138.000000,40.605975,-106.821875
max,21754.000000,7300.000000,4425.000000,162.000000,549.000000,28.000000,47.000000,283.00000,180.000000,0.986000,0.095100,1116.000000,1647.000000,43.680800,-105.573000


### Create skiiing dataframe from skitowns dataframe

In [5]:
skidata = skitownsdf[['Town', 'Total Skiable Acreage', 'Ski In/Out Acreage', 'Vertical', 'Lifts', 'Avg Snowfall']]
# This data fram has only the skiiing related data so we can compare the towns and derive a neumerical rating for skiing for each town
skidata

,Town,Total Skiable Acreage,Ski In/Out Acreage,Vertical,Lifts,Avg Snowfall
0,Aspen,3362,673,4406,38,300
1,Crested Butte,1547,1547,3055,16,300
2,Incline Village,21754,655,4139,162,400
3,Jackson,4909,2500,3100,54,459
4,Park City,9326,7300,3668,62,355
5,Steamboat Springs,3006,2956,3400,20,349
6,Sun Valley,2154,2154,3500,18,220
7,Taos,1494,1294,3281,20,300
8,Telluride,2000,2000,4425,18,309
9,Vail,9611,5289,3450,79,549


In [6]:
#Calculate Lifts per Acre values and add to dataframe
skidata2 = skidata.copy()
skidata2['Lifts per Acre'] = (skidata.iloc[: , 4]/skidata.iloc[: , 1]).copy()

skidata2

,Town,Total Skiable Acreage,Ski In/Out Acreage,Vertical,Lifts,Avg Snowfall,Lifts per Acre
0,Aspen,3362,673,4406,38,300,0.011303
1,Crested Butte,1547,1547,3055,16,300,0.010343
2,Incline Village,21754,655,4139,162,400,0.007447
3,Jackson,4909,2500,3100,54,459,0.011000
4,Park City,9326,7300,3668,62,355,0.006648
5,Steamboat Springs,3006,2956,3400,20,349,0.006653
6,Sun Valley,2154,2154,3500,18,220,0.008357
7,Taos,1494,1294,3281,20,300,0.013387
8,Telluride,2000,2000,4425,18,309,0.009000
9,Vail,9611,5289,3450,79,549,0.008220


In [7]:
#Normalizing the sking data from 0-1 for each column.

from sklearn import preprocessing
x = skidata2.iloc[: , 1:7].values.astype(float)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
normalized_skidata = pd.DataFrame(x_scaled, columns=skidata2.iloc[:,1:7].columns)
normalized_skidata

,Total Skiable Acreage,Ski In/Out Acreage,Vertical,Lifts,Avg Snowfall,Lifts per Acre
0,0.092201,0.002709,0.986131,0.150685,0.243161,0.690734
1,0.002616,0.134236,0.000000,0.000000,0.243161,0.548246
2,1.000000,0.000000,0.791241,1.000000,0.547112,0.118541
3,0.168559,0.277652,0.032847,0.260274,0.726444,0.645831
4,0.386575,1.000000,0.447445,0.315068,0.410334,0.000000
5,0.074630,0.346275,0.251825,0.027397,0.392097,0.000783
6,0.032577,0.225583,0.324818,0.013699,0.000000,0.253527
7,0.000000,0.096163,0.164964,0.027397,0.243161,1.000000
8,0.024975,0.202408,1.000000,0.013699,0.270517,0.349012
9,0.400642,0.697366,0.288321,0.431507,1.000000,0.233227


### Here we derive a final normalized and weighted score for the skiing component of the total score

In [8]:
#Double the weight of Total Skiable Acres then sum all the normalized scores to derive a final skiing score for each town.
normalized_skidata.insert(0, 'Town', ['Aspen', 'Crested Butte', 'Incline Village', 'Jackson', 'Park City', 'Steamboat Springs', 'Sun Valley', 'Taos','Telluride', 'Vail'])
normalized_skidata['Total Skiable Acreage'] = normalized_skidata['Total Skiable Acreage']*2
normalized_skidata['Skiing Score'] = normalized_skidata.sum(axis = 1)
normalized_skidata

,Town,Total Skiable Acreage,Ski In/Out Acreage,Vertical,Lifts,Avg Snowfall,Lifts per Acre,Skiing Score
0,Aspen,0.184403,0.002709,0.986131,0.150685,0.243161,0.690734,2.257823
1,Crested Butte,0.005232,0.134236,0.000000,0.000000,0.243161,0.548246,0.930875
2,Incline Village,2.000000,0.000000,0.791241,1.000000,0.547112,0.118541,4.456895
3,Jackson,0.337117,0.277652,0.032847,0.260274,0.726444,0.645831,2.280165
4,Park City,0.773149,1.000000,0.447445,0.315068,0.410334,0.000000,2.945997
5,Steamboat Springs,0.149260,0.346275,0.251825,0.027397,0.392097,0.000783,1.167638
6,Sun Valley,0.065153,0.225583,0.324818,0.013699,0.000000,0.253527,0.882779
7,Taos,0.000000,0.096163,0.164964,0.027397,0.243161,1.000000,1.531684
8,Telluride,0.049951,0.202408,1.000000,0.013699,0.270517,0.349012,1.885585
9,Vail,0.801283,0.697366,0.288321,0.431507,1.000000,0.233227,3.451704


### Create Quality of Life (QoL) dataframe from skitowns dataframe

In [9]:
QoLdf = skitownsdf[['Town', 'Avg Jan Temp','Air Quality Index', 'Avg An Sunny Days', 'Cost of Living Index', 'School Quality', 'Crime Rate' ]]
QoLdf

,Town,Avg Jan Temp,Air Quality Index,Avg An Sunny Days,Cost of Living Index,School Quality,Crime Rate
0,Aspen,21,41,246,140,0.917,0.0426
1,Crested Butte,13,41,273,153,0.986,0.0178
2,Incline Village,26,44,251,172,0.939,0.0111
3,Jackson,14,40,212,144,0.962,0.0185
4,Park City,28,47,229,180,0.804,0.0353
5,Steamboat Springs,19,43,242,139,0.936,0.0247
6,Sun Valley,14,13,204,145,0.806,0.0123
7,Taos,24,15,283,104,0.600,0.0951
8,Telluride,26,43,242,168,0.959,0.0238
9,Vail,17,42,247,149,0.567,0.0459


In [10]:
#Low numbers are better for Air Quality Index, Cost of Living Index and Crime Rate so they need to be inverted before normalizing
QoLInvdf = QoLdf.copy()
QoLInvdf.iloc[:, 2] = 1/QoLdf.iloc[:, 2]
QoLInvdf.iloc[:, 4] = 1/QoLdf.iloc[:, 4]
QoLInvdf.iloc[:, 6] = 1/QoLdf.iloc[:, 6]

QoLInvdf

,Town,Avg Jan Temp,Air Quality Index,Avg An Sunny Days,Cost of Living Index,School Quality,Crime Rate
0,Aspen,21,0.024390,246,0.007143,0.917,23.474178
1,Crested Butte,13,0.024390,273,0.006536,0.986,56.179775
2,Incline Village,26,0.022727,251,0.005814,0.939,90.090090
3,Jackson,14,0.025000,212,0.006944,0.962,54.054054
4,Park City,28,0.021277,229,0.005556,0.804,28.328612
5,Steamboat Springs,19,0.023256,242,0.007194,0.936,40.485830
6,Sun Valley,14,0.076923,204,0.006897,0.806,81.300813
7,Taos,24,0.066667,283,0.009615,0.600,10.515247
8,Telluride,26,0.023256,242,0.005952,0.959,42.016807
9,Vail,17,0.023810,247,0.006711,0.567,21.786492


In [11]:
#Normalize the QoL data
x = QoLInvdf.iloc[: , 1:7].values.astype(float)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
normalized_QoLdata = pd.DataFrame(x_scaled, columns=QoLInvdf.iloc[:,1:7].columns)
normalized_QoLdata

,Avg Jan Temp,Air Quality Index,Avg An Sunny Days,Cost of Living Index,School Quality,Crime Rate
0,0.533333,0.055954,0.531646,0.390977,0.835322,0.162852
1,0.000000,0.055954,0.873418,0.241486,1.000000,0.573856
2,0.866667,0.026070,0.594937,0.063647,0.887828,1.000000
3,0.066667,0.066912,0.101266,0.342105,0.942721,0.547143
4,1.000000,0.000000,0.316456,0.000000,0.565632,0.223857
5,0.400000,0.035568,0.481013,0.403635,0.880668,0.376634
6,0.066667,1.000000,0.000000,0.330309,0.570406,0.889547
7,0.733333,0.815686,1.000000,1.000000,0.078759,0.000000
8,0.866667,0.035568,0.481013,0.097744,0.935561,0.395873
9,0.266667,0.045518,0.544304,0.284705,0.000000,0.141643


### Here we derive a final normalized and weighted score for the skiing component of the total score

In [12]:
#Assign a weight of 2 to the normalized Crime Weight and sum the normalized values to derive an over all Quality of Life Score
normalized_QoLdata.insert(0, 'Town', ['Aspen', 'Crested Butte', 'Incline Village', 'Jackson', 'Park City', 'Steamboat Springs', 'Sun Valley', 'Taos','Telluride', 'Vail'])
normalized_QoLdata['Crime Rate'] = normalized_QoLdata['Crime Rate']*2
normalized_QoLdata['Quality of Life Score'] = normalized_QoLdata.sum(axis = 1)
normalized_QoLdata

,Town,Avg Jan Temp,Air Quality Index,Avg An Sunny Days,Cost of Living Index,School Quality,Crime Rate,Quality of Life Score
0,Aspen,0.533333,0.055954,0.531646,0.390977,0.835322,0.325704,2.672937
1,Crested Butte,0.000000,0.055954,0.873418,0.241486,1.000000,1.147713,3.318571
2,Incline Village,0.866667,0.026070,0.594937,0.063647,0.887828,2.000000,4.439149
3,Jackson,0.066667,0.066912,0.101266,0.342105,0.942721,1.094286,2.613956
4,Park City,1.000000,0.000000,0.316456,0.000000,0.565632,0.447713,2.329802
5,Steamboat Springs,0.400000,0.035568,0.481013,0.403635,0.880668,0.753268,2.954151
6,Sun Valley,0.066667,1.000000,0.000000,0.330309,0.570406,1.779094,3.746475
7,Taos,0.733333,0.815686,1.000000,1.000000,0.078759,0.000000,3.627779
8,Telluride,0.866667,0.035568,0.481013,0.097744,0.935561,0.791747,3.208299
9,Vail,0.266667,0.045518,0.544304,0.284705,0.000000,0.283287,1.424480


## Get venue information from foursquare and put in dataframe 

In [13]:
#Define Foursquare credentials
CLIENT_ID = 'YRBYYRJPWHDQDBOGBXA02Y2R03T40N5SY2DZNPT5WS4OW4NK' # your Foursquare ID
CLIENT_SECRET = 'SLUMLBHW1ZQXOPZKYXSR0MZFF0MB3V4CHV4BOTCSAT3ZGXJM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [14]:
#get venues and place in a data frame
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    LIMIT = 200
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Town', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
skitown_venues = getNearbyVenues(names=skitownsdf['Town'],
                                   latitudes=skitownsdf['Latitude'],
                                   longitudes=skitownsdf['Longitude']
                                  )
print(skitown_venues.shape)
skitown_venues.head(10)

(764, 7)


,Town,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Aspen,39.1906,-106.8186,Hotel Jerome,39.190912,-106.819413,Hotel
1,Aspen,39.1906,-106.8186,meat & cheese,39.189990,-106.820369,Wine Bar
2,Aspen,39.1906,-106.8186,White House Tavern,39.190244,-106.820494,Whisky Bar
3,Aspen,39.1906,-106.8186,J-Bar at Hotel Jerome,39.190837,-106.819489,Hotel Bar
4,Aspen,39.1906,-106.8186,Matsuhisa,39.190691,-106.820330,Japanese Restaurant
5,Aspen,39.1906,-106.8186,Cache Cache Restaurant,39.189772,-106.820083,French Restaurant
6,Aspen,39.1906,-106.8186,Mi Chola,39.190492,-106.819183,Mexican Restaurant
7,Aspen,39.1906,-106.8186,Pyramid Bistro,39.190720,-106.821113,Vegetarian / Vegan Restaurant
8,Aspen,39.1906,-106.8186,Library Bar @ Hotel Jerome,39.190895,-106.819390,Cocktail Bar
9,Aspen,39.1906,-106.8186,Hooch,39.189774,-106.820420,Cocktail Bar


### Put foursquare data into dataframe

In [16]:
# one hot encoding
ski_onehot = pd.get_dummies(skitown_venues[['Venue Category']], prefix="", prefix_sep="")

# add town column back to dataframe
ski_onehot['Town'] = skitown_venues['Town'] 

# move town column to the first column
fixed_columns = [ski_onehot.columns[-1]] + list(ski_onehot.columns[:-1])
ski_onehot = ski_onehot[fixed_columns]

ski_onehot.head(25)

,Town,ATM,Accessories Store,American Restaurant,Amphitheater,Apres Ski Bar,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop
0,Aspen,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Aspen,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,Aspen,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Aspen,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Aspen,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Aspen,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Aspen,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Aspen,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
8,Aspen,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Aspen,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
ski_onehot.shape

(764, 168)

In [18]:
#Group venues by type in each town.
venues_grouped = ski_onehot.groupby('Town').sum().reset_index()
venues_grouped.head(12)

,Town,ATM,Accessories Store,American Restaurant,Amphitheater,Apres Ski Bar,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop
0,Aspen,0,0,6,0,0,0,1,0,0,...,0,0,1,5,1,0,0,1,2,0
1,Crested Butte,0,0,3,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,Incline Village,0,0,4,0,0,0,0,0,0,...,1,1,0,1,0,0,0,0,0,0
3,Jackson,0,0,6,0,0,1,0,0,1,...,0,1,0,1,1,0,0,0,0,0
4,Park City,1,1,6,1,1,1,0,0,2,...,0,0,0,0,0,0,0,0,0,0
5,Steamboat Springs,0,0,6,0,0,0,0,0,0,...,0,1,0,1,0,1,1,0,0,0
6,Sun Valley,1,0,5,0,0,0,0,0,1,...,1,0,0,2,0,0,0,0,1,0
7,Taos,0,0,4,0,0,0,1,1,0,...,0,0,1,0,0,0,0,0,0,1
8,Telluride,0,0,4,0,0,2,0,0,0,...,0,0,0,2,0,0,0,0,0,0
9,Vail,0,0,9,0,0,0,0,0,0,...,0,0,0,2,0,0,0,1,1,0


In [19]:
#Examine the top 10 most common venues in each town

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Town']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
skitown_venues_sorted = pd.DataFrame(columns=columns)
skitown_venues_sorted['Town'] = venues_grouped['Town']

for ind in np.arange(venues_grouped.shape[0]):
    skitown_venues_sorted.iloc[ind, 1:] = return_most_common_venues(venues_grouped.iloc[ind, :], num_top_venues)

skitown_venues_sorted

,Town,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aspen,Hotel,Italian Restaurant,American Restaurant,Trail,Ski Area,Sandwich Place,Cocktail Bar,Park,Restaurant,Coffee Shop
1,Crested Butte,American Restaurant,Steakhouse,Pizza Place,Bar,Coffee Shop,Breakfast Spot,Mexican Restaurant,Burger Joint,Brewery,Gastropub
2,Incline Village,American Restaurant,Beach,Coffee Shop,Golf Course,Deli / Bodega,Pizza Place,Hotel,Bar,Brewery,BBQ Joint
3,Jackson,Hotel,American Restaurant,BBQ Joint,Bar,Resort,Coffee Shop,Mexican Restaurant,Clothing Store,Breakfast Spot,Ski Lodge
4,Park City,American Restaurant,Coffee Shop,Ski Area,Mexican Restaurant,Café,Sporting Goods Shop,Bar,Ski Lodge,Sushi Restaurant,Asian Restaurant
5,Steamboat Springs,American Restaurant,Pizza Place,Coffee Shop,Café,Italian Restaurant,Brewery,Mexican Restaurant,Pub,Candy Store,Park
6,Sun Valley,Hotel,American Restaurant,Ski Chairlift,Bar,Sporting Goods Shop,Clothing Store,Italian Restaurant,New American Restaurant,Café,Ski Area
7,Taos,Mexican Restaurant,American Restaurant,Hotel,Coffee Shop,Brewery,Grocery Store,Bar,Fast Food Restaurant,Breakfast Spot,Pizza Place
8,Telluride,Hotel,American Restaurant,Bar,Ski Area,Sandwich Place,Pizza Place,Sporting Goods Shop,Indie Movie Theater,Italian Restaurant,Art Gallery
9,Vail,Hotel,American Restaurant,Ski Area,Resort,Pizza Place,French Restaurant,Trail,Steakhouse,Sandwich Place,Bar


In [22]:
#Add Total column for each town and put sum accross line in that column.
venues_grouped['Total'] = venues_grouped.sum(axis = 1)
venues_grouped.head()

,Town,ATM,Accessories Store,American Restaurant,Amphitheater,Apres Ski Bar,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Tourist Information Center,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Total
0,Aspen,0,0,6,0,0,0,1,0,0,...,0,1,5,1,0,0,1,2,0,100
1,Crested Butte,0,0,3,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,36
2,Incline Village,0,0,4,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,63
3,Jackson,0,0,6,0,0,1,0,0,1,...,1,0,1,1,0,0,0,0,0,98
4,Park City,1,1,6,1,1,1,0,0,2,...,0,0,0,0,0,0,0,0,0,100


In [23]:
list(venues_grouped.columns)

['Town',
 'ATM',
 'Accessories Store',
 'American Restaurant',
 'Amphitheater',
 'Apres Ski Bar',
 'Art Gallery',
 'Art Museum',
 'Arts & Crafts Store',
 'Asian Restaurant',
 'Athletics & Sports',
 'BBQ Joint',
 'Bagel Shop',
 'Bakery',
 'Bank',
 'Bar',
 'Beach',
 'Bed & Breakfast',
 'Beer Bar',
 'Beer Garden',
 'Bike Rental / Bike Share',
 'Bike Shop',
 'Board Shop',
 'Boat or Ferry',
 'Bookstore',
 'Boutique',
 'Bowling Alley',
 'Breakfast Spot',
 'Brewery',
 'Buffet',
 'Burger Joint',
 'Burrito Place',
 'Bus Station',
 'Café',
 'Candy Store',
 'Casino',
 'Chinese Restaurant',
 'Chocolate Shop',
 'Climbing Gym',
 'Clothing Store',
 'Cocktail Bar',
 'Coffee Shop',
 'Concert Hall',
 'Convenience Store',
 'Cosmetics Shop',
 'Coworking Space',
 'Creperie',
 'Deli / Bodega',
 'Department Store',
 'Dessert Shop',
 'Diner',
 'Discount Store',
 'Distillery',
 'Dive Bar',
 'Dog Run',
 'Event Space',
 'Farmers Market',
 'Fast Food Restaurant',
 'Field',
 'Fishing Store',
 'Fondue Restaurant',


## Seperate each broad category of venue into its own dataframe and get total for each category in each town.

There are many different key words to describe a restautrant so we will lump anything that is like a restaurant into a single restaurant array. For example a restaurant could also be listed as a joint, spot, cafe, bistro, etc.

We will then do the same for the broad categories of shopping, entertainment and recreation. All other venues that don't fit into any of those 4 categories are lumped into an additional category called "Other".

In [24]:
#Create dataframe of all restaurants
restaurantsdf = venues_grouped.loc[:, venues_grouped.columns.str.contains('Restaurant|Joint|Bistro|Spot|Buffet|Place|Caf*|Coffee|Creperie|Deli|Diner|Gastropub|Steakhouse')].copy()
restaurantsdf.insert(0, 'Town', ['Aspen', 'Crested Butte', 'Incline Village', 'Jackson', 'Park City', 'Steamboat Springs', 'Sun Valley', 'Taos','Telluride', 'Vail'])



In [25]:
restaurantsdf['Restaurant Total'] = restaurantsdf.sum(axis=1)
restaurantsdf

,Town,American Restaurant,Asian Restaurant,BBQ Joint,Breakfast Spot,Buffet,Burger Joint,Burrito Place,Café,Candy Store,...,Southern / Soul Food Restaurant,Steakhouse,Sushi Restaurant,Swiss Restaurant,Taco Place,Tapas Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Restaurant Total
0,Aspen,6,0,1,1,0,0,0,2,0,...,0,1,1,0,0,0,0,1,0,41
1,Crested Butte,3,1,0,2,0,1,1,0,0,...,0,2,1,0,0,0,0,0,0,23
2,Incline Village,4,0,2,0,0,0,0,0,0,...,0,0,0,0,0,2,1,0,0,27
3,Jackson,6,1,4,3,0,2,0,1,0,...,0,0,1,0,0,1,2,1,0,38
4,Park City,6,2,0,1,1,0,0,4,0,...,0,2,2,0,0,1,0,0,0,43
5,Steamboat Springs,6,0,1,1,0,1,0,3,2,...,1,1,1,0,0,0,0,0,1,36
6,Sun Valley,5,1,0,1,0,1,1,2,1,...,0,2,1,0,0,0,0,0,0,31
7,Taos,4,0,0,2,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,29
8,Telluride,4,0,1,0,0,1,0,2,0,...,0,1,0,0,1,0,1,0,0,21
9,Vail,9,0,1,0,0,0,0,0,0,...,0,2,1,1,0,0,0,0,0,36


In [26]:
#can't explain presence of "Candy Store" or "Casino" droping from df
restaurantsdf.drop(['Candy Store', 'Casino'], axis =1, inplace = True)
restaurantsdf

,Town,American Restaurant,Asian Restaurant,BBQ Joint,Breakfast Spot,Buffet,Burger Joint,Burrito Place,Café,Chinese Restaurant,...,Southern / Soul Food Restaurant,Steakhouse,Sushi Restaurant,Swiss Restaurant,Taco Place,Tapas Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Restaurant Total
0,Aspen,6,0,1,1,0,0,0,2,1,...,0,1,1,0,0,0,0,1,0,41
1,Crested Butte,3,1,0,2,0,1,1,0,0,...,0,2,1,0,0,0,0,0,0,23
2,Incline Village,4,0,2,0,0,0,0,0,1,...,0,0,0,0,0,2,1,0,0,27
3,Jackson,6,1,4,3,0,2,0,1,1,...,0,0,1,0,0,1,2,1,0,38
4,Park City,6,2,0,1,1,0,0,4,0,...,0,2,2,0,0,1,0,0,0,43
5,Steamboat Springs,6,0,1,1,0,1,0,3,0,...,1,1,1,0,0,0,0,0,1,36
6,Sun Valley,5,1,0,1,0,1,1,2,0,...,0,2,1,0,0,0,0,0,0,31
7,Taos,4,0,0,2,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,29
8,Telluride,4,0,1,0,0,1,0,2,0,...,0,1,0,0,1,0,1,0,0,21
9,Vail,9,0,1,0,0,0,0,0,0,...,0,2,1,1,0,0,0,0,0,36


In [27]:
# Create dataframe of all shopping venues
shoppingdf = venues_grouped.loc[:, venues_grouped.columns.str.contains('Gallery|Store|Shop|Bookstore|Boutique|Market|Organic|Pharmacy|Mall')].copy()
shoppingdf.insert(0, 'Town', ['Aspen', 'Crested Butte', 'Incline Village', 'Jackson', 'Park City', 'Steamboat Springs', 'Sun Valley', 'Taos','Telluride', 'Vail'])


In [28]:
#Sum shopping venues of all types for each town
shoppingdf['Shopping Total'] = shoppingdf.sum(axis=1)
shoppingdf

,Town,Accessories Store,Art Gallery,Arts & Crafts Store,Bagel Shop,Bike Shop,Board Shop,Bookstore,Boutique,Candy Store,...,Paper / Office Supplies Store,Pharmacy,Shipping Store,Smoothie Shop,Sporting Goods Shop,Thrift / Vintage Store,Toy / Game Store,Video Store,Wine Shop,Shopping Total
0,Aspen,0,0,0,0,0,1,1,1,0,...,0,0,1,0,2,0,1,0,0,12
1,Crested Butte,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
2,Incline Village,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,9
3,Jackson,0,1,0,1,1,0,1,0,0,...,1,1,1,0,1,0,0,0,0,22
4,Park City,1,1,0,0,0,1,1,1,0,...,0,1,0,1,3,0,0,0,0,24
5,Steamboat Springs,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,1,0,10
6,Sun Valley,0,0,0,0,0,0,1,0,1,...,0,0,1,0,3,1,0,0,0,17
7,Taos,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,13
8,Telluride,0,2,0,0,1,0,1,0,0,...,0,0,0,0,2,0,0,0,0,8
9,Vail,0,0,0,0,0,2,0,0,0,...,0,0,0,0,1,0,0,0,0,10


In [29]:
#Create dataframe of all recreation venues
recreationdf = venues_grouped.loc[:, venues_grouped.columns.str.contains('Beach|Bike|Bowling|Gym|Golf|Field|Marina|Arena|Lake|Park|Pool|Rafting|Club|Rink|Spa|Stables|Tennis')].copy()
recreationdf.insert(0, 'Town', ['Aspen', 'Crested Butte', 'Incline Village', 'Jackson', 'Park City', 'Steamboat Springs', 'Sun Valley', 'Taos','Telluride', 'Vail'])


In [30]:
#Sum all recreation venues for each town.
recreationdf['Rec Total'] = recreationdf.sum(axis=1)
recreationdf

,Town,Beach,Bike Rental / Bike Share,Bike Shop,Bowling Alley,Climbing Gym,Coworking Space,Event Space,Field,Golf Course,...,Rafting,Rock Club,Skate Park,Skating Rink,Spa,Sports Club,Stables,Tennis Court,Theme Park Ride / Attraction,Rec Total
0,Aspen,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,5
1,Crested Butte,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Incline Village,4,0,1,1,1,0,1,0,2,...,0,0,0,0,0,0,0,0,0,14
3,Jackson,0,0,1,0,0,0,0,0,0,...,2,1,0,0,0,0,0,0,0,7
4,Park City,0,0,0,0,0,1,0,0,0,...,0,0,0,0,2,0,0,0,2,7
5,Steamboat Springs,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,5
6,Sun Valley,0,0,0,1,0,0,0,1,1,...,0,0,1,2,1,0,1,0,0,14
7,Taos,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
8,Telluride,0,0,1,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,4
9,Vail,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,1,0,1,0,6


In [31]:
#Create dataframe of all entertainment venues
entertainmentdf = venues_grouped.loc[:, venues_grouped.columns.str.contains('Amphitheater|Casino|Hall|Event|Entertainment|Theater|Venue|Racetrack|Ride')].copy()
entertainmentdf.insert(0, 'Town', ['Aspen', 'Crested Butte', 'Incline Village', 'Jackson', 'Park City', 'Steamboat Springs', 'Sun Valley', 'Taos','Telluride', 'Vail'])


In [32]:
entertainmentdf['Entertainment Total'] = entertainmentdf.sum(axis=1)
entertainmentdf

,Town,Amphitheater,Casino,Concert Hall,Event Space,General Entertainment,Indie Movie Theater,Movie Theater,Music Venue,Performing Arts Venue,Racetrack,Theater,Theme Park Ride / Attraction,Entertainment Total
0,Aspen,0,0,2,0,0,0,0,0,1,0,1,0,4
1,Crested Butte,0,0,0,0,0,0,1,0,1,0,0,0,2
2,Incline Village,0,1,0,1,0,0,0,0,0,0,0,0,2
3,Jackson,0,0,0,0,0,1,0,0,1,1,0,0,3
4,Park City,1,0,0,0,1,0,1,0,0,0,1,2,6
5,Steamboat Springs,0,0,0,0,0,0,0,0,0,0,0,1,1
6,Sun Valley,0,0,0,0,0,0,2,1,0,0,0,0,3
7,Taos,0,0,0,0,0,0,0,0,0,0,0,0,0
8,Telluride,0,0,0,0,0,2,1,1,1,0,0,0,5
9,Vail,0,0,1,0,1,0,0,1,0,0,0,0,3


In [33]:
#Create new dataframe containing the total venues by broad category for each town.
Venuesdf = pd.DataFrame([restaurantsdf['Town'], restaurantsdf['Restaurant Total'], shoppingdf['Shopping Total'], recreationdf['Rec Total'], entertainmentdf['Entertainment Total']],).copy().transpose()
Venuesdf                                                

,Town,Restaurant Total,Shopping Total,Rec Total,Entertainment Total
0,Aspen,41,12,5,4
1,Crested Butte,23,5,0,2
2,Incline Village,27,9,14,2
3,Jackson,38,22,7,3
4,Park City,43,24,7,6
5,Steamboat Springs,36,10,5,1
6,Sun Valley,31,17,14,3
7,Taos,29,13,1,0
8,Telluride,21,8,4,5
9,Vail,36,10,6,3


In [34]:
Venuesdf['subtotal'] = Venuesdf.iloc[: , 1:5].sum(axis=1)
#Derive the values for the "other" venues category buy subtracting the Total of the 4 previously defined categories from the all venues total


In [35]:
tempdf = pd.DataFrame([venues_grouped['Total'], Venuesdf['subtotal']]).copy().transpose()
tempdf

,Total,subtotal
0,100.0,62.0
1,36.0,30.0
2,63.0,52.0
3,98.0,70.0
4,100.0,80.0
5,61.0,52.0
6,100.0,65.0
7,57.0,43.0
8,56.0,38.0
9,93.0,55.0


In [36]:
Venuesdf['Other'] = tempdf['Total'] - tempdf['subtotal']


In [37]:
Venuesdf.drop(['subtotal'], axis=1, inplace=True)
Venuesdf

,Town,Restaurant Total,Shopping Total,Rec Total,Entertainment Total,Other
0,Aspen,41,12,5,4,38.0
1,Crested Butte,23,5,0,2,6.0
2,Incline Village,27,9,14,2,11.0
3,Jackson,38,22,7,3,28.0
4,Park City,43,24,7,6,20.0
5,Steamboat Springs,36,10,5,1,9.0
6,Sun Valley,31,17,14,3,35.0
7,Taos,29,13,1,0,14.0
8,Telluride,21,8,4,5,18.0
9,Vail,36,10,6,3,38.0


In [38]:
#Normalize the Venue data
x = Venuesdf.iloc[: , 1:6].values.astype(float)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
normalized_Venuedata = pd.DataFrame(x_scaled, columns=Venuesdf.iloc[:,1:6].columns)
normalized_Venuedata

,Restaurant Total,Shopping Total,Rec Total,Entertainment Total,Other
0,0.909091,0.368421,0.357143,0.666667,1.00000
1,0.090909,0.000000,0.000000,0.333333,0.00000
2,0.272727,0.210526,1.000000,0.333333,0.15625
3,0.772727,0.894737,0.500000,0.500000,0.68750
4,1.000000,1.000000,0.500000,1.000000,0.43750
5,0.681818,0.263158,0.357143,0.166667,0.09375
6,0.454545,0.631579,1.000000,0.500000,0.90625
7,0.363636,0.421053,0.071429,0.000000,0.25000
8,0.000000,0.157895,0.285714,0.833333,0.37500
9,0.681818,0.263158,0.428571,0.500000,1.00000


In [39]:
#Lots of Restaurants are a sign of a vibrant local economy. I like to be active so other forms of recreation are important to me during non-ski season. So assigning a weight of 2 to each of those brings the total poosible points for venues
#to 7 matching skiing and Quality of Life.
normalized_Venuedata.insert(0, 'Town', ['Aspen', 'Crested Butte', 'Incline Village', 'Jackson', 'Park City', 'Steamboat Springs', 'Sun Valley', 'Taos','Telluride', 'Vail'])
normalized_Venuedata['Restaurant Total'] = normalized_Venuedata['Restaurant Total']*2
normalized_Venuedata['Rec Total'] = normalized_Venuedata['Rec Total']*2
normalized_Venuedata['Venue Score'] = normalized_Venuedata.sum(axis = 1)
normalized_Venuedata

,Town,Restaurant Total,Shopping Total,Rec Total,Entertainment Total,Other,Venue Score
0,Aspen,1.818182,0.368421,0.714286,0.666667,1.00000,4.567555
1,Crested Butte,0.181818,0.000000,0.000000,0.333333,0.00000,0.515152
2,Incline Village,0.545455,0.210526,2.000000,0.333333,0.15625,3.245564
3,Jackson,1.545455,0.894737,1.000000,0.500000,0.68750,4.627691
4,Park City,2.000000,1.000000,1.000000,1.000000,0.43750,5.437500
5,Steamboat Springs,1.363636,0.263158,0.714286,0.166667,0.09375,2.601497
6,Sun Valley,0.909091,0.631579,2.000000,0.500000,0.90625,4.946920
7,Taos,0.727273,0.421053,0.142857,0.000000,0.25000,1.541183
8,Telluride,0.000000,0.157895,0.571429,0.833333,0.37500,1.937657
9,Vail,1.363636,0.263158,0.857143,0.500000,1.00000,3.983937


## Arrive at final total score for each town

In [40]:
#Create new dataframe for just the 3 main scoring categories
skitown_ratings = pd.DataFrame([normalized_Venuedata['Town'], normalized_Venuedata['Venue Score'], normalized_QoLdata['Quality of Life Score'], normalized_skidata['Skiing Score']]).copy().transpose()


In [41]:
skitown_ratings['Total Score'] = skitown_ratings.iloc[:, 1:4].sum(axis=1)
skitown_ratings.sort_values(by='Total Score', ascending=False,axis=0,inplace=True)

skitown_ratings

,Town,Venue Score,Quality of Life Score,Skiing Score,Total Score
2,Incline Village,3.24556,4.43915,4.45689,12.141607
4,Park City,5.4375,2.3298,2.946,10.713299
6,Sun Valley,4.94692,3.74648,0.882779,9.576174
3,Jackson,4.62769,2.61396,2.28016,9.521812
0,Aspen,4.56756,2.67294,2.25782,9.498315
9,Vail,3.98394,1.42448,3.4517,8.860122
8,Telluride,1.93766,3.2083,1.88559,7.031541
5,Steamboat Springs,2.6015,2.95415,1.16764,6.723286
7,Taos,1.54118,3.62778,1.53168,6.700645
1,Crested Butte,0.515152,3.31857,0.930875,4.764597


### Finally, let's compare the Total Score above to price of real estate in each town to find where the best value is to be found. 

In [42]:
#Median Hove value column from initial dataframe and derive best value investment
#skitown_ratings['Median Home Value (000s)'] = skitownsdf['Median Home Value (000s)']
skitown_ratings['Median Home Price per sq ft'] = skitownsdf['Median Home Price per sq ft']

In [43]:
skitown_ratings['Value per Home $'] = (skitown_ratings.iloc[:, 4]*1000/skitown_ratings.iloc[:, 5])
skitown_ratings.sort_values(by='Value per Home $', ascending=False,axis=0,inplace=True)
skitown_ratings

,Town,Venue Score,Quality of Life Score,Skiing Score,Total Score,Median Home Price per sq ft,Value per Home $
7,Taos,1.54118,3.62778,1.53168,6.700645,226,29.648874
2,Incline Village,3.24556,4.43915,4.45689,12.141607,533,22.779751
6,Sun Valley,4.94692,3.74648,0.882779,9.576174,498,19.229265
4,Park City,5.4375,2.3298,2.946,10.713299,652,16.431440
5,Steamboat Springs,2.6015,2.95415,1.16764,6.723286,429,15.671995
3,Jackson,4.62769,2.61396,2.28016,9.521812,823,11.569638
1,Crested Butte,0.515152,3.31857,0.930875,4.764597,521,9.145100
9,Vail,3.98394,1.42448,3.4517,8.860122,1243,7.128014
0,Aspen,4.56756,2.67294,2.25782,9.498315,1647,5.767040
8,Telluride,1.93766,3.2083,1.88559,7.031541,1380,5.095320


## Lets look at a map with venues for the highest rated town, Incline Village, NV.

In [44]:
#Get dataframe of Incline Village venues from original dataframe of all venues in all towns.
incline_venues = skitown_venues[skitown_venues['Town'] == 'Incline Village']

incline_venues

,Town,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
136,Incline Village,39.2497,-119.9525,IV Coffee Lab,39.249221,-119.951731,Coffee Shop
137,Incline Village,39.2497,-119.9525,Alibi Ale Works,39.247492,-119.953652,Brewery
138,Incline Village,39.2497,-119.9525,Mofo's Pizza & Pasta,39.250136,-119.954198,Pizza Place
139,Incline Village,39.2497,-119.9525,Bite Restaurant & Bar,39.249421,-119.951716,Tapas Restaurant
140,Incline Village,39.2497,-119.9525,T's Mesquite Rotisserie,39.249668,-119.952087,BBQ Joint
141,Incline Village,39.2497,-119.9525,Crosby's Grill Pub & Casino,39.250199,-119.955615,Bar
142,Incline Village,39.2497,-119.9525,Raley's,39.250625,-119.949667,Grocery Store
143,Incline Village,39.2497,-119.9525,Koi Sushi,39.250259,-119.955613,Japanese Restaurant
144,Incline Village,39.2497,-119.9525,Village Pub,39.247527,-119.956759,Dive Bar
145,Incline Village,39.2497,-119.9525,Alibi Ale Works Incline Public House,39.249100,-119.948549,Brewery


In [45]:
# create map of Incline Village using latitude and longitude values
import folium
latitude = skitownsdf.iloc[2,14]
longitude = skitownsdf.iloc[2,15]
map_Incline_Village = folium.Map(location=[latitude, longitude], zoom_start=14)


# add markers to map
for lat, lng, category , venue in zip(incline_venues['Venue Latitude'], incline_venues['Venue Longitude'], incline_venues['Venue Category'], incline_venues['Venue']):
    label = '{}, {}'.format(venue, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Incline_Village)  

map_Incline_Village